In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install tensorflow-text
import tensorflow_text as text
import tensorflow as tf
import tensorflow_hub as hub
import pandas as pd
import numpy as np

from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.8 MB 9.0 MB/s 
     |████████████████████████████████| 588.3 MB 18 kB/s 
     |████████████████████████████████| 439 kB 54.2 MB/s 
     |████████████████████████████████| 1.7 MB 44.4 MB/s 
     |████████████████████████████████| 6.0 MB 32.3 MB/s 
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.9.0
    Uninstalling tensorflow-estimator-2.9.0:
      Successfully uninstalled tensorflow-estimator-2.9.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.9.1
    Uninstalling tensorboard-2.9.1:
      Successfully uninstalled tensorboard-2.9.1
  Attempting uninstall: keras
    Found existing installation: keras 2.9.0
    Uninstalling keras-2.9.0:
      Successfully uninstalled keras-2.9.0
  Attempting uninstall: flatbuffers
    Found existing installation: flatbuffers 1.1

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/data_for_colab/tripadvisor_hotel_reviews.csv')

df = df[df.Rating != 3]

df['is_positive'] = (df['Rating'] >= 4).astype(int)

In [ ]:
df.drop(columns=['Rating'], inplace=True)

In [ ]:
df.columns = ['DATA_COLUMN', 'LABEL_COLUMN']
df

,DATA_COLUMN,LABEL_COLUMN
0,nice hotel expensive parking got good deal sta...,1
1,ok nothing special charge diamond member hilto...,0
3,"unique, great stay, wonderful time hotel monac...",1
4,"great stay great stay, went seahawk game aweso...",1
5,love monaco staff husband stayed hotel crazy w...,1
...,...,...
20486,"best kept secret 3rd time staying charm, not 5...",1
20487,great location price view hotel great quick pl...,1
20488,"ok just looks nice modern outside, desk staff ...",0
20489,hotel theft ruined vacation hotel opened sept ...,0


In [ ]:
df['LABEL_COLUMN'].value_counts()

1    15093
0     3214
Name: LABEL_COLUMN, dtype: int64

In [ ]:
df_positive = df[df['LABEL_COLUMN']==1]

In [ ]:
df_negative = df[df['LABEL_COLUMN']==0]

In [ ]:
# Тестовая выборка

In [ ]:
n_test = (df_negative.shape[0] // 4) * 3 # в оригинале (df_negative.shape[0] // 4) * 3
df_negative_test = df_negative.tail(n_test)
n_test = (df_positive.shape[0] // 20) * 3 # в оригинале df_positive.shape[0] // 20) * 3
df_positive_test = df_positive.tail(n_test)

In [ ]:
df_positive_test

,DATA_COLUMN,LABEL_COLUMN
17565,wonderful hotel wonderful town unbeknownst man...,1
17566,wonderful wonderful wonderful just returned ye...,1
17567,"affinia 50 you'all, stayed affinia 50 week, li...",1
17568,just right good location walking distance bloo...,1
17569,loved place want thank outstanding service goi...,1
...,...,...
20480,great play stay stay loyal inn package deal ha...,1
20482,great choice wife chose best western quite bit...,1
20483,good bed clean convenient just night happy sta...,1
20486,"best kept secret 3rd time staying charm, not 5...",1


In [ ]:
df_balanced_test = pd.concat([df_negative_test, df_positive_test])

In [ ]:
df_balanced_test.sample(10)

,DATA_COLUMN,LABEL_COLUMN
20233,great hotel stay extremely pleased hotel accom...,1
19170,sophistocated hotel perfect location hotel loc...,1
15631,little things wonderful stay harbour view hong...,0
20248,hotel des academies et des arts hidden gem tim...,1
19190,great hotel waikiki hotel renew relaxing sanct...,1
18397,"magnificent, just returned trip toronto canada...",1
4151,consider staying lobby absolutely beautiful lo...,0
20429,"positives good things hotel, staff friendly he...",1
19022,best hotel stayed europe stayed hotel 4 nights...,1
19876,loved hotel location staff bedroom stayed hote...,1


In [ ]:
df_balanced_test['LABEL_COLUMN'].value_counts()

0    2409
1    2262
Name: LABEL_COLUMN, dtype: int64

In [ ]:
# Обучающая выборка

In [ ]:
n_train = df_negative.shape[0] // 4 # в оригианле df_negative.shape[0] // 4
df_negative_train = df_negative.head(n_train)
n_train = df_positive.shape[0] // 20 # в оригинале df_positive.shape[0] // 20
df_positive_train = df_positive.head(n_train)

In [ ]:
df_balanced_train = pd.concat([df_negative_train, df_positive_train])

In [ ]:
df_balanced_train['LABEL_COLUMN'].value_counts()

0    803
1    754
Name: LABEL_COLUMN, dtype: int64

In [ ]:
df_balanced_train.sample(10)

,DATA_COLUMN,LABEL_COLUMN
353,university inn good place agree consensus revi...,1
1688,expectations house power outage decided spend ...,0
712,"awesome place stay, not beat location, looking...",1
907,romantic rex rex perfect hotel enjoy romantic ...,1
1942,fortune fortune house fortune house condo hote...,0
2327,"not par hotel reopened 3 weeks ago storm, thin...",0
2233,bumped hotel sister property hotel saint louis...,0
700,"hated, disappointed hotel.when reservations mo...",0
2109,fawlty towers usa style sister property 215 w....,0
315,town visitors love hotel town visitors love ol...,1


In [ ]:
X_train = df_balanced_train['DATA_COLUMN'].squeeze()
y_train = df_balanced_train['LABEL_COLUMN'].squeeze()

In [ ]:
X_test = df_balanced_test['DATA_COLUMN'].squeeze()
y_test = df_balanced_test['LABEL_COLUMN'].squeeze()

In [ ]:
electra_preprocess = hub.KerasLayer('https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3') # uses same thing as bert



In [ ]:
electra_encoder = hub.KerasLayer('https://tfhub.dev/google/electra_base/2')

In [ ]:
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
preprocessed_text = electra_preprocess(text_input)
outputs = electra_encoder(preprocessed_text)

Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


In [ ]:
l = tf.keras.layers.Dropout(0.1, name="dropout")(outputs['pooled_output'])
l = tf.keras.layers.Dense(1, activation='sigmoid', name="output")(l)


In [ ]:
model = tf.keras.Model(inputs=[text_input], outputs = [l])

In [ ]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 text (InputLayer)              [(None,)]            0           []                               
                                                                                                  
 keras_layer (KerasLayer)       {'input_word_ids':   0           ['text[0][0]']                   
                                (None, 128),                                                      
                                 'input_type_ids':                                                
                                (None, 128),                                                      
                                 'input_mask': (Non                                               
                                e, 128)}                                                      

In [ ]:
METRICS = [
      tf.keras.metrics.BinaryAccuracy(name='accuracy'),
      tf.keras.metrics.Precision(name='precision'),
      tf.keras.metrics.Recall(name='recall')
]

model.compile(optimizer='adam',
 loss='binary_crossentropy',
 metrics=METRICS)

In [ ]:
model.fit(X_train, y_train, epochs=10)

Epoch 1/10
49/49 [==============================] - 672s 13s/step - loss: 0.6475 - accuracy: 0.6320 - precision: 0.6291 - recall: 0.5849
Epoch 2/10
49/49 [==============================] - 652s 13s/step - loss: 0.5758 - accuracy: 0.7206 - precision: 0.7308 - recall: 0.6698
Epoch 3/10
49/49 [==============================] - 649s 13s/step - loss: 0.5331 - accuracy: 0.7669 - precision: 0.7653 - recall: 0.7480
Epoch 4/10
49/49 [==============================] - 646s 13s/step - loss: 0.5078 - accuracy: 0.7752 - precision: 0.7837 - recall: 0.7401
Epoch 5/10
49/49 [==============================] - 646s 13s/step - loss: 0.4883 - accuracy: 0.7797 - precision: 0.7858 - recall: 0.7493
Epoch 6/10
49/49 [==============================] - 648s 13s/step - loss: 0.4703 - accuracy: 0.7977 - precision: 0.8105 - recall: 0.7599
Epoch 7/10
49/49 [==============================] - 643s 13s/step - loss: 0.4568 - accuracy: 0.8009 - precision: 0.8058 - recall: 0.7759
Epoch 8/10
49/49 [=======================

In [ ]:
y_predicted = model.predict(X_test)
y_predicted


146/146 [==============================] - 1957s 13s/step


array([[0.410001  ],
       [0.3564024 ],
       [0.13383779],
       ...,
       [0.8195089 ],
       [0.5344264 ],
       [0.9524873 ]], dtype=float32)

In [ ]:
y_predicted = np.where(y_predicted > 0.5, 1, 0)
y_predicted

array([[0],
       [0],
       [0],
       ...,
       [1],
       [1],
       [1]])

In [ ]:
y_test

4133     0
4134     0
4135     0
4136     0
4140     0
        ..
20480    1
20482    1
20483    1
20486    1
20487    1
Name: LABEL_COLUMN, Length: 4671, dtype: int64

In [ ]:
y_test_array = np.array(y_test)

In [ ]:
accuracy_score(y_test, y_predicted)

0.7767073431813316

In [ ]:
precision_score(y_test_array, y_predicted)


0.7644251626898048

In [ ]:
recall_score(y_test_array, y_predicted)

0.7789566755083996

In [ ]:
f1_score(y_test_array, y_predicted)

0.7716225093058902

In [ ]:
df_results_for_reviews_on_hotels = pd.DataFrame(columns=['accuracy', 'precision', 'recall', 'f1_score'])

In [ ]:
df_results_for_reviews_on_hotels.loc['reviews on hotels', 'accuracy'] = accuracy_score(y_test, y_predicted)
df_results_for_reviews_on_hotels.loc['reviews on hotels', 'precision'] = precision_score(y_test, y_predicted)
df_results_for_reviews_on_hotels.loc['reviews on hotels', 'recall'] = recall_score(y_test, y_predicted)
df_results_for_reviews_on_hotels.loc['reviews on hotels', 'f1_score'] =  f1_score(y_test_array, y_predicted)


In [ ]:
saved_model_path = '/content/drive/MyDrive/data_for_colab/electra_trained_on_hotel_reviews_24_december'

In [ ]:
print(saved_model_path)

/content/drive/MyDrive/data_for_colab/electra_trained_on_hotel_reviews_24_december


In [ ]:
model.save(saved_model_path, include_optimizer=True) 

In [ ]:
df_results_for_reviews_on_hotels


,accuracy,precision,recall,f1_score
reviews on hotels,0.776707,0.764425,0.778957,0.771623
